In [1]:
from datasets import Dataset
import json

# Load the dataset from a JSON file
with open('../data/corpus/train.json', 'r') as f:
    data = json.load(f)

d = {
    'slot': [item['slot'] for item in data],
    'text': [item['text'] for item in data],
    'position': [item['positions'] for item in data]
}


# create dataset from dict (train split)
dataset = Dataset.from_dict(d)
# 

/home/pedro/opensource/piured-search/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = dataset.train_test_split(test_size=0.025)

In [3]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [4]:
def classes(data):
    l = []
    for item in data:
        l.extend(item['slot'].keys())

    list_set = set(l)
    length = len(list_set)
    return list(list_set), length

In [5]:

class_labels, no_classes = classes(data)
print(class_labels)
print(no_classes)

['artist', 'credit', 'bpm_lower_than', 'pump_routine', 'warps', 'fakes', 'pump_halfdouble', 'scrolls', 'pump_single', 'mix', 'stops', 'meter_greater_than', 'tune', 'pump_double', 'speeds', 'bpm_greater_than', 'meter_lower_than', 'pump_couple', 'meter', 'bpm']
20


In [6]:
def class_mapper(class_labels):
    no_classes = len(class_labels)
    d = {}
    # 0 is reserved for no class
    d['O'] = 0
    for i in range(1, no_classes + 1):
        d[class_labels[i-1]] = i
    
    # and another dictionary to map the index to the class label

    d_reverse = {}
    d_reverse[0] = 'O'
    for i in range(1, no_classes + 1):
        d_reverse[i] = class_labels[i-1]

    return d, d_reverse

mapper, unmapper = class_mapper(class_labels)
print(mapper, unmapper)


{'O': 0, 'artist': 1, 'credit': 2, 'bpm_lower_than': 3, 'pump_routine': 4, 'warps': 5, 'fakes': 6, 'pump_halfdouble': 7, 'scrolls': 8, 'pump_single': 9, 'mix': 10, 'stops': 11, 'meter_greater_than': 12, 'tune': 13, 'pump_double': 14, 'speeds': 15, 'bpm_greater_than': 16, 'meter_lower_than': 17, 'pump_couple': 18, 'meter': 19, 'bpm': 20} {0: 'O', 1: 'artist', 2: 'credit', 3: 'bpm_lower_than', 4: 'pump_routine', 5: 'warps', 6: 'fakes', 7: 'pump_halfdouble', 8: 'scrolls', 9: 'pump_single', 10: 'mix', 11: 'stops', 12: 'meter_greater_than', 13: 'tune', 14: 'pump_double', 15: 'speeds', 16: 'bpm_greater_than', 17: 'meter_lower_than', 18: 'pump_couple', 19: 'meter', 20: 'bpm'}


In [7]:
def align_positions_with_tokens(input_ids, offset_mapping, position):
    begin_end_tokens = [(offset_mapping[i][0], offset_mapping[i][1])  for i, token in enumerate(input_ids)]
    labels = [ 0 for i in range(len(input_ids))]
    labels[0] = -100
    # find index of SEP token (102)
    sep_index = input_ids.index(102)
    # from sep_index to the end of the list, set the label to -100
    for i in range(sep_index, len(labels)):
        labels[i] = -100

    
    begin_end_tokens = begin_end_tokens[1:-1]
    for key, val in position.items():
        if (val != None):
            begin_gt = val['begin']
            end_gt = val['end']

            class_label = key


            # find the indices of the tokens that contain the begin and end of the ground truth
            begin_token = [i for i, token in enumerate(begin_end_tokens) if token[0] >= begin_gt and token[0] < end_gt ]
            end_token = [i for i, token in enumerate(begin_end_tokens) if token[1] == end_gt ]


            # create list of indices of tokens that are part of the ground truth
            try: 
                tokens = [i for i in range(begin_token[0], end_token[0]+1)]
            except Exception as e:
                print(begin_end_tokens)
                print(f'begin: {begin_gt}, end: {end_gt}, class: {class_label}')
                print(f'begin_token: {begin_token}, end_token: {end_token}')
                print('---')
                raise e

            # set the label of the tokens that are part of the ground truth
            for token in tokens:
                labels[token + 1] = mapper[class_label]
    
    return labels

sample = dataset['train'][12239]
inputs = tokenizer(sample["text"], is_split_into_words=False, return_offsets_mapping=True)
print(sample['text'])
print(inputs.tokens())
labels = align_positions_with_tokens(inputs['input_ids'], inputs['offset_mapping'], sample['position'])
print(labels)


    

album EXCEED~ZE
['[CLS]', 'album', 'exceed', '~', 'ze', '[SEP]']
[-100, 0, 10, 10, 10, -100]


In [8]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["text"], is_split_into_words=False, return_offsets_mapping=True, truncation=True, padding=True, max_length=48
    )
    new_labels = []
    for i, position in enumerate(examples['position']):
        try:
            new_labels.append(align_positions_with_tokens(tokenized_inputs['input_ids'][i], tokenized_inputs['offset_mapping'][i], position))
        except Exception as e:
            print( examples['text'][i])
            print(tokenized_inputs.tokens(i))
            print(f'index: {i}')
            raise e

    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [9]:
tokenized_datasets = dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=["slot", "text", "position"],
)

Map: 100%|██████████| 2500/2500 [00:00<00:00, 14671.39 examples/s]


In [10]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [11]:
import evaluate

metric = evaluate.load("seqeval")

In [12]:
def decode_labels(labels):
    return [unmapper[label] if label != -100 else 'O' for label in labels]

In [13]:

decoded_labels = decode_labels(tokenized_datasets['train']["labels"][0])
print(decoded_labels)
pred = decoded_labels.copy()
pred[1] = 'bpm_greater_than'

metric.compute(predictions=[pred], references=[decoded_labels], )

['O', 'stops', 'stops', 'O', 'O', 'meter_lower_than', 'O', 'O', 'O', 'bpm_greater_than', 'tune', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


/home/pedro/opensource/piured-search/.venv/lib/python3.11/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: stops seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/pedro/opensource/piured-search/.venv/lib/python3.11/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: meter_lower_than seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/pedro/opensource/piured-search/.venv/lib/python3.11/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: bpm_greater_than seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/pedro/opensource/piured-search/.venv/lib/python3.11/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: tune seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))


{'eter_lower_than': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'pm_greater_than': {'precision': 0.5,
  'recall': 1.0,
  'f1': 0.6666666666666666,
  'number': 1},
 'tops': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1},
 'une': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 0.6,
 'overall_recall': 0.75,
 'overall_f1': 0.6666666666666665,
 'overall_accuracy': 0.9629629629629629}

In [14]:
import numpy as np
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[ unmapper[label] for label in sample if label != -100] for sample in labels]
    true_predictions = [
        [unmapper[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [15]:
from transformers import DistilBertForTokenClassification 

model = DistilBertForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=unmapper,
    label2id=mapper,
)

model.config.num_labels = no_classes + 1


Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
from transformers import TrainingArguments

args = TrainingArguments(
    "distilbert-piu-search",
    evaluation_strategy='steps',
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    eval_steps=500,
    per_device_train_batch_size=16,
)

In [17]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

  3%|▎         | 500/18282 [00:14<07:59, 37.08it/s]

{'loss': 0.4065, 'learning_rate': 1.9453013893447107e-05, 'epoch': 0.08}


/home/pedro/opensource/piured-search/.venv/lib/python3.11/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: speeds seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/pedro/opensource/piured-search/.venv/lib/python3.11/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: pump_routine seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/pedro/opensource/piured-search/.venv/lib/python3.11/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: meter_greater_than seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/pedro/opensource/piured-search/.venv/lib/python3.11/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: bpm_lower_than seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/pedro/opensource/piured-search/.venv/lib/python3.11/site-packages/seqeval/metrics/sequence_label

{'eval_loss': 0.05714792385697365, 'eval_precision': 0.9706809151938621, 'eval_recall': 0.976433296582139, 'eval_f1': 0.9735486087255237, 'eval_accuracy': 0.9834633092173259, 'eval_runtime': 1.4685, 'eval_samples_per_second': 1702.375, 'eval_steps_per_second': 213.137, 'epoch': 0.08}


  5%|▌         | 1000/18282 [00:30<08:08, 35.35it/s]

...

{'loss': 0.0097, 'learning_rate': 1.2889180614812384e-05, 'epoch': 1.07}


/home/pedro/opensource/piured-search/.venv/lib/python3.11/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: speeds seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/pedro/opensource/piured-search/.venv/lib/python3.11/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: pump_routine seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/pedro/opensource/piured-search/.venv/lib/python3.11/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: meter_greater_than seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/pedro/opensource/piured-search/.venv/lib/python3.11/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: meter_lower_than seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/pedro/opensource/piured-search/.venv/lib/python3.11/site-packages/seqeval/metrics/sequence_lab

{'eval_loss': 0.012142508290708065, 'eval_precision': 0.9946280991735538, 'eval_recall': 0.9951764057331863, 'eval_f1': 0.9949021769082392, 'eval_accuracy': 0.9967584327727145, 'eval_runtime': 1.5616, 'eval_samples_per_second': 1600.903, 'eval_steps_per_second': 200.433, 'epoch': 1.07}


 38%|███▊      | 6913/18282 [03:31<05:03, 37.47it/s]

KeyboardInterrupt: 

In [18]:
# load model from disk checkpoint
model_cpu = DistilBertForTokenClassification.from_pretrained(
    "distilbert-piu-search/checkpoint-6094",
)

In [19]:
from transformers import pipeline
token_classifier = pipeline(
    "token-classification", model=model_cpu, tokenizer=tokenizer, aggregation_strategy="simple"
)


In [20]:

pred = token_classifier("EXC d 20", )
print(pred)


[{'entity_group': 'mix', 'score': 0.9676077, 'word': 'exc', 'start': 0, 'end': 3}, {'entity_group': 'pump_double', 'score': 0.9950138, 'word': 'd', 'start': 4, 'end': 5}, {'entity_group': 'meter', 'score': 0.9997477, 'word': '20', 'start': 6, 'end': 8}]


 38%|███▊      | 6914/18282 [03:45<05:03, 37.47it/s]

In [ ]:
[ { "class": p['entity_group'], "word": p['word'], "start": p['start'], "end": p['end']} for p in pred]